# Dictionnaire de rimes

Nous allons essayer de construire un dictionnaire de rimes à partir de la ressource [lexique 3.80](http://www.lexique.org/).
À l'aide de la fonction 'recherche par propriété' nous avons pu extraire les mots portant la catégorie 'NOM' du lexique et récupérer pour chacun d'entre les informations suivantes : ortho, phon, lemme, cgram, genre, nombre

Dans le fichier `noms-lexique.org.txt` nous avons (5 premières lignes) :

In [3]:
with open('noms-lexique.org.txt', 'r') as f:
    for i in range(0, 5):
        line = f.readline()
        line = line.rstrip()
        print(line)

ortho	phon	lemme	cgram	genre	nombre
a	a	a	NOM	m
a priori	apRijoRi	a priori	NOM	m
à-côté	akote	à-côté	NOM	m	s
à-côtés	akote	à-côté	NOM	m	p


Notre dictionnaire de rimes devra *in fine* permettre de trouver les rimes possibles d'un mot c-a-d les noms qui ont la même terminaison phonétique.
Pour cela nous devrons conserver deux informations du fichier : ortho et phon  
Commençons pour ça.

 ## Extraction des données

Nous allons parcourir l'intégralité du fichier et ne récupérer que les informations ortho et phon  
C'est un fichier au format `tsv`, les champs sont séparés par une tabulation.

In [4]:
words = []
with open('noms-lexique.org.txt', 'r') as f:
    f.readline() # première ligne
    for line in f:
        fields = line.split('\t')
        ortho = fields[0]
        phon = fields[1]
        words.append((ortho, phon))            
          

##### On peut synthétiser un peu le code à l'aide des tranches

In [5]:
words = []
with open('noms-lexique.org.txt', 'r') as f:
    f.readline() # première ligne
    for line in f:
        ortho, phon = line.split('\t')[0:2]
        words.append((ortho, phon))
words[3:6]

[('à-côtés', 'akote'), ('à-coup', 'aku'), ('à-coups', 'aku')]

Hop, on emballe dans une fonction :

In [8]:
def extract(filename):
    """Extrait du fichier arguement les deux premiers champs
    arg : nom du fichier au format tsv
    return : list de tuples (ortho, phon)
    """
    words = []
    with open(filename, 'r') as f:
        f.readline() # première ligne
        for line in f:
            ortho, phon = line.split('\t')[0:2]
            words.append((ortho, phon))
    return words

In [9]:
lexique = extract('noms-lexique.org.txt')
lexique[:5]

[('a', 'a'),
 ('a priori', 'apRijoRi'),
 ('à-côté', 'akote'),
 ('à-côtés', 'akote'),
 ('à-coup', 'aku')]

## Constitution de la structure de données

Nous voulons classer les noms de notre liste en fonction des phonèmes de terminaison.  
Et cela selon trois classements différents : par suite de 3 phonèmes, 2 phonèmes et enfin un phonème.  

In [10]:
dico_3 = {}
for item in lexique:
    if len(item[1]) >= 3:
        rime_3 = item[1][-3:]
        #if rime_3 not in dico_3:
        #    dico_3[rime_3] = list()
        #dico_3[rime_3].append(item[0])
        dico_3.setdefault(rime_3, []).append(item[0])
        

In [11]:
def mk_dico(lexique, n):
    """
    Construit un dictionnaire de rimes de longueur n
    à partir d'un lexique phonétisé
    args : lexique [(ortho, phon)], n int
    return : dict {rime : [word1, word2, ..]}
    """
    dico = {}
    for item in lexique:
        if len(item[1]) >= n:
            rime = item[1][-n:]
            dico.setdefault(rime, []).append(item[0])
    return dico

## Parcours et recherche dans la structure de données

In [12]:
dico_3 = mk_dico(lexique, 3)
dico_2 = mk_dico(lexique, 2)

Maintenant que nous avons constitué notre dictionnaire des rimes nous allons nous en servir.
Le scénario d'utilisation est le suivant, pour un mot donné :  
1. trouver la transcription phonétique
2. extraire de la transcription les 3 derniers phonèmes (ou 2 le cas échéant)
3. trouver dans le dictionnaire la liste des mots du lexique qui ont la même suite de phonèmes finaux
4. piocher un mot au hasard dans la liste

In [13]:
import random

def ortho2phon(word, words_list):
    """
    Trouve un mot (word) dans une liste (words_list)
    et retourne la forme phonétique correspondante
    (en cas d'homographe non homophone, retourne le premier trouvé)
    args : word (str), words_list [(ortho, phon), (.., ..)]
    return : str, "" si word ne fait pas partie de la liste
    """
    for item in words_list:
        if word == item[0]:
            return item[1]
    return ""

def find_rhyme(word, dico, lexique, n=3):
    """
    Pour un mot donné, retourne un mot au hasard dont les n
    derniers phonèmes riment
    args : word (str), dico (dict) le dictionnaire de rimes, 
        lexique (list) lexique ortho, phon, n (int) le nombre de phonèmes terminaux
    """
    # 1 trouver la transcription phonétique
    phon = ortho2phon(word, lexique)
    if not phon:
        return None
    # 2 extraire de la transcription les 3 derniers phonèmes (ou 2 le cas échéant)
    # 3 trouver dans le dictionnaire la liste des mots du lexique qui ont la même suite de phonèmes finaux
    if phon[-n:] not in dico:
        return None
    rhymes = dico[phon[-n:]]
    if word in rhymes:
        rhymes.remove(word)
    # 4. piocher un mot au hasard dans la liste
    rand = random.randint(0, len(rhymes) - 1)
    return rhymes[rand]

In [14]:
word = "livres"
rime = find_rhyme(word, dico_3, lexique)
print(rime)

savoir-vivre


## Exemple

Nous allons utiliser notre dictionnaire de rimes pour modifier la première strophe de *Brise Marine* de Mallarmé :  

La chair est triste, hélas ! et j'ai lu tous les livres.  
Fuir ! là-bas fuir! Je sens que des oiseaux sont ivres  
D'être parmi l'écume inconnue et les cieux !  
Rien, ni les vieux jardins reflétés par les yeux  
Ne retiendra ce coeur qui dans la mer se trempe  
Ô nuits ! ni la clarté déserte de ma lampe  
Sur le vide papier que la blancheur défend  
Et ni la jeune femme allaitant son enfant.  
Je partirai ! Steamer balançant ta mâture,  
Lève l'ancre pour une exotique nature !  



L'idée est remplacer le dernier mot de chaque vers par un mot pioché au hasard dans notre dictionnaire de rimes.
On prendra d'abord un mot dans la liste des rimes à 3 phonèmes puis une deuxième version à 2 phonèmes.

In [24]:
with open('brise-marine.txt', 'r') as f:
    for line in f:
        line = line.rstrip()
        if line == "":
            continue
        words = line.split(" ")
        # traitement ad hoc des fins de ligne
        # pour l'exercice ça ira mais c'est moche
        if words[-1] == '!':
            word = words[-2]
        else:
            word = words[-1]
        if word[-1] == '.' or word[-1] == ',':
            word = word[:-1]
        rhyme = find_rhyme(word, dico_3, lexique)
        if rhyme:
            print(line.replace(word, rhyme))
        else:
            print(line)


La chair est triste, hélas ! et j’ai lu tous les savoir-vivre.
Fuir ! là-bas fuir! Je sens que des oiseaux sont ivres
D’être parmi l’écume inconnue et les vicieux !
Rien, ni les vieux jardins reflétés par les yeux
Ne retiendra ce coeur qui dans la mer se crampe
Ô nuits ! ni la clarté déserte de ma clamp
Sur le vide papier que la blancheur défend
Et ni la jeune femme allaitant son mi-enfant.
Je partirai ! Steamer balançant ta statures,
Lève l’ancre pour une exotique cultures !
Un Ennui, désolé par les cruels auditoire,
Croit encore à l’adieu suprême des ivoire !
Et, peut-être, les mâts, invitant les entourages,
Sont-ils de ceux qu’un vent penche sur les affacturage
Perdus, sans mâts, sans mâts, ni fertiles silo
Mais, ô mon coeur, entends le chant des angelots !
